In [1]:
import pandas as pd

df = pd.read_json("matches/Kai'sa_matches.json")

df

,metadata,info
0,"{'dataVersion': '2', 'matchId': 'NA1_530516583...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
1,"{'dataVersion': '2', 'matchId': 'NA1_530506101...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
2,"{'dataVersion': '2', 'matchId': 'NA1_530438224...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
3,"{'dataVersion': '2', 'matchId': 'NA1_530435374...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
4,"{'dataVersion': '2', 'matchId': 'NA1_530386744...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
5,"{'dataVersion': '2', 'matchId': 'EUN1_37908077...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
6,"{'dataVersion': '2', 'matchId': 'EUN1_37907954...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
7,"{'dataVersion': '2', 'matchId': 'EUN1_37907755...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
8,"{'dataVersion': '2', 'matchId': 'EUN1_37907573...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
9,"{'dataVersion': '2', 'matchId': 'EUN1_37907309...","{'endOfGameResult': 'GameComplete', 'gameCreat..."
